### RAG 구현을 위한 단계로 벡터DB 최적화된 모듈 사용
1. 일반적을 pandas와 numpy를 이용함
2. 실무에서는 아래와 같은 모듈을 사용함

OpenAI API 키 값을 세팅하기 위한 os, 파일을 다운로드하기 위한 urllib.request를 임포트하고, 실 습을 위해 필요한 랭체인 도구들을 임포트합니다. 앞서 학습했던 PDF 를 로드하는 PyPDFLoader, 문서 들을 다수의 청크로 분할하는 RecursiveCharacterTextSplitter, 청크들을 임베딩 벡터로 변 환 시 OpenAI 의 Embedding API 를 사용하기 위해 OpenAIEmbeddings, 임베딩 벡터들을 적재하기 위한 벡터 데이터베이스인 Chroma와 Faiss를 임포트하고, 사용자의 OpenAI API 키 값을 현재 실습 환 경에 세팅합니다

In [4]:
import os
import urllib.request
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

In [ ]:
from dotenv import load_dotenv

load_dotenv()
openAPI_KEY = os.getenv('openAPI')

os.environ['OPENAI_API_KEY'] = openAPI_KEY

### 데이터베이스 불러오고

In [6]:
loader = PyPDFLoader('2023_북한인권보고서.pdf')
pages = loader.load_and_split()
print("청크의 수: ", len(pages))

청크의 수:  445


### 추가로 청크하고

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [8]:
splitted_docs = text_splitter.split_documents(pages)
print('분할된 청크의 수 :', len(splitted_docs))

분할된 청크의 수 : 496


In [9]:
chunks = [splitted_doc.page_content for splitted_doc in splitted_docs]
print('청크의 최대 길이 :',max(len(chunk) for chunk in chunks))
print('청크의 최소 길이  :',min(len(chunk) for chunk in chunks))
print('청크의 평균 길이  :',sum(map(len, chunks))/len(chunks))

청크의 최대 길이 : 1000
청크의 최소 길이  : 6
청크의 평균 길이  : 750.2983870967741


### 임베딩과 크로마 데이터베이스에 적재

In [10]:
# 하나의 함수에서 임베딩과 동시에 수행
db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100))
print("문서의 수: ", db._collection.count())

문서의 수:  496


### 유사 내용 찾기 실습
1. 자체적으로 similarity_serarch함수를 제공한다.
2. 이 함수가 많은 일을 하는 것 같네.
3. 질문 내용을 임베딩 처리도 하나부네

In [11]:
question = "북한의 교육"
docs = db.similarity_search(question)
print("문서의 수: ", len(docs))

문서의 수:  4


In [13]:
# 위에서 찾는 문서를 출력해보자
for doc in docs:
    print(doc)
    print("##" * 50)

page_content='2023 북한인권보고서
184
데, 당국이 실시하는 반종교 교육을 통해 기독교를 접한 경우였다. 기
독교 관련 북한당국의 반종교 교육은 학교 교과과정에서 뿐만 아니
라 졸업 후 조직생활을 통해서도 이루어지고 있었다. 수집된 증언에 
따르면 북한에서 반종교 교육을 받고 종교에 대한 부정적 인식이 증
가했다고 한다. 기독교를 믿는 사람을 반동분자로 인식하고 있었다
는 증언들도 있었다. 한 증언자는 2015년에 계급교양관을 1달에 1번
씩 참관해야 했는데, 거기서 ‘종교는 침략자들이 북한에 가져온 것으
로 그들이 성경도 가져왔다’는 내용이 포함된 반종교 교육을 받았다
고 한다. 기독교인들은 제국주의적 침략의 앞잡이이므로 반민족적·
반혁명적 적대계층이라는 내용도 있었다고 한다. 다른 증언자는 교육
기관, 사회기관, 법기관에서 ‘종교는 허황된 것이고 거짓’이며 선교사
는 악한 자라고 세뇌가 될 정도로 지속적으로 교육하기 때문에 감히 
종교에 관심을 가질 생각도 하지 못했고 ‘선교사’라는 단어를 들으면 
지금도 무섭다고 한다. 인민반 강연 등에서 기독교를 믿는 사람은 반
동분자라고 하면서 이들을 보면 신고하라고 했다는 증언도 있었다. 
“주민들을 모아 놓고 교양하는 때가 있었는데, 그 때 처음 ‘성경책 
사진, 예배하는 모습의 사진’을 본 적이 있습니다. 반동조직이라고 
교육하면서 성경에는 미신에 관한 것이 적혀있어 이것을 읽게 되면 
사상이 변질되니 이런 책을 주변에서 보게 되면 신고하라고 했습니
다. 이들은 국가 반역자로 이 땅에서 사라져야 한다고 선전했습니
다. 그래서 재북시 저 뿐만이 아니라 북한 주민들은 성경책이 북한 
제도에 대해 안 좋게 적어놓은 책으로 알고 있었고 이를 소지하면 
죽임을 당한다고 알고 있었습니다. ”
북한에서 기독교를 탄압하는 이유는 기독교의 유일신 사상이 수
령 우상화 정책과 주체사상에 반하기 때문이라고 한다. 한 증언자는' metadata={'total_pages': 448, 'source': '2023_북한인권

In [14]:
# 크로마 db를 저장하는 것도 가능. 실무에서는 이렇게 사용하지 않을까
db_to_file = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100) , persist_directory = './chroma_test.db')
print('문서의 수 :', db_to_file._collection.count())

문서의 수 : 496


In [15]:
# 저장한 db를 로드해서 사용해보자
db_from_file = Chroma(persist_directory = './chroma_test.db', embedding_function=OpenAIEmbeddings())
print('문서의 수: ', db_from_file._collection.count())

문서의 수:  496


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_107228\970855786.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db_from_file = Chroma(persist_directory = './chroma_test.db', embedding_function=OpenAIEmbeddings())


앞서 similarity_search(사용자의 입력)을 사용했을 때는 사용자의 입력에 대해 유사한 청크 4 개를 찾아냈습니다. 내부적으로는 유사도를 구하고 유사도 점수 상위 4 개의 청크를 찾아낸 것입니다. 이번에는 유사한 청크를 상위 3 개만 찾도록 강제하고, 유사도 점수 또한 출력하도록 해보겠습니다. 그러려면 similarity_search_with_relevance_scores(사용자의 입력, k=찾고자 하는
문서의  수)와 같이 하면 됩니다. 유사한 청크를 상위 3 개만 찾도록 강제하기 위해 k 의 값을 3 으로 지 정했고 유사도 점수 상위 3 개의 청크를 찾아서 출력합니다.

In [16]:
# 유사한 내용의 개수제한 및 추가 정보 출력
question = "북한의 교육 과정"
top_three_docs = db_from_file.similarity_search_with_relevance_scores(question, k=3)
for doc in top_three_docs:
    print(doc)
    print("##" * 50)

(Document(metadata={'source': '2023_북한인권보고서.pdf', 'page': 41, 'producer': 'Adobe PDF Library 10.0.1', 'creationdate': '2023-07-31T13:50:27+09:00', 'trapped': '/False', 'moddate': '2023-07-31T13:57:54+09:00', 'creator': 'Adobe InDesign CS6 (Windows)', 'page_label': '42', 'total_pages': 448}, page_content='2023 북한인권보고서\n40\n명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과\n서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교\n과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학\n년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교\n부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 \n수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 \n비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 \n의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 \n앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그\n만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육\n환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교\n육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제\n력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, \n학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 \n없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절\n히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모\n의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 \n제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다

### 임베딩과 파이스 db 적재

In [17]:
faiss_db = FAISS.from_documents(splitted_docs, OpenAIEmbeddings())
print('문서의 수 :', faiss_db.index.ntotal)

문서의 수 : 496


In [18]:
# 파이스 벡터 db 적재, 불러오기
faiss_db.save_local("faiss_index")

new_db_faiss = FAISS.load_local("faiss_index",
                                OpenAIEmbeddings(),
                                allow_dangerous_deserialization=True)

In [19]:
question = "북한의 교육 과정"

docs = new_db_faiss.similarity_search(question)

for doc in docs:
    print(doc)
    print("#" * 100)

page_content='2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-

In [20]:
question = "북한의 교육 과정"

docs = new_db_faiss.similarity_search_with_relevance_scores(question, k=3)

for doc in docs:
    print(doc)
    print("#" * 100)

(Document(id='93a67195-6e18-4612-a5b9-8c44624ccd3a', metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-31T13:50:27+09:00', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False', 'source': '2023_북한인권보고서.pdf', 'total_pages': 448, 'page': 41, 'page_label': '42'}, page_content='2023 북한인권보고서\n40\n명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과\n서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교\n과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학\n년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교\n부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 \n수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 \n비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 \n의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 \n앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그\n만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육\n환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교\n육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제\n력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, \n학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 \n없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절\n히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모\n의 원조를 받거나 자신의 텃밭에 학생을 동원시키고